# 抓取江苏省政协十年提案

打开http://www.jszx.gov.cn/zxta/2019ta/

- 点击下一页，url不变!

> 所以数据的更新是使用js推送的
- 分析network中的内容，发现proposalList.jsp
    - 查看它的header，并发现了form_data
    
<img src = './img/form_data.png'>

http://www.jszx.gov.cn/zxta/2019ta/

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
form_data = {'year':2019,
        'pagenum':1,
        'pagesize':20
}
url = 'http://www.jszx.gov.cn/wcm/zxweb/proposalList.jsp'
content = requests.get(url, form_data)
content.encoding = 'utf-8'
js = content.json()

In [3]:
js['data']['totalcount']

'424'

In [4]:
dat = js['data']['list']
pagenum = js['data']['pagecount']

In [5]:
for i in range(2, pagenum+1):
    print(i)
    form_data['pagenum'] = i
    content = requests.get(url, form_data)
    content.encoding = 'utf-8'
    js = content.json()
    for j in js['data']['list']:
        dat.append(j)

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22


In [6]:
len(dat)

424

In [7]:
dat[0]

{'personnel_name': '邹正',
 'pkid': '18b1b347f9e34badb8934c2acec80e9e',
 'proposal_number': '0001',
 'publish_time': '2019-01-12 16:04:48',
 'reason': '关于完善城市环卫公厕指引系统的建议',
 'rownum': 1,
 'type': '城乡建设',
 'year': '2019'}

In [8]:
import pandas as pd

df = pd.DataFrame(dat)
df.head()

,personnel_name,pkid,proposal_number,publish_time,reason,rownum,type,year
0,邹正,18b1b347f9e34badb8934c2acec80e9e,0001,2019-01-12 16:04:48,关于完善城市环卫公厕指引系统的建议,1,城乡建设,2019
1,省政协学习委员会,da43aae2378244faa961dd1224d1343e,0002,2019-01-12 16:04:48,关于加强老小区光纤化改造的建议,2,城乡建设,2019
2,许文前,c0a1626a1bb744ebb0852cf25b21fb0a,0004,2019-01-12 15:42:19,加强科技创新，推动制造业转型升级,3,工业商贸,2019
3,段绪强,ce60d71296764cfe997d62bb2c0990af,0005,2019-01-12 16:21:46,关于落实金融政策、促进民营企业高质量发展的建议,4,财税金融,2019
4,侯建军,8b5fb5a7d86547899835a12af398ffc7,0006,2019-01-12 15:42:19,关于主基地航空公司协同东部机场集团发展的建议,5,工业商贸,2019


In [9]:
df.groupby('type').size()

type
农林水利     3
医卫体育    45
城乡建设    25
工业商贸    34
政治建设    15
教育事业    58
文化宣传    34
法制建设    24
社会事业    75
科学技术    24
经济发展    47
统战综合     4
财税金融    12
资源环境    24
dtype: int64

## 抓取提案内容
http://www.jszx.gov.cn/zxta/2019ta/index_61.html?pkid=18b1b347f9e34badb8934c2acec80e9e

In [10]:
url_base = 'http://www.jszx.gov.cn/wcm/zxweb/proposalInfo.jsp?pkid='
urls = [url_base + i  for i in df['pkid']]

In [11]:
import sys
def flushPrint(www):
    sys.stdout.write('\r')
    sys.stdout.write('%s' % www)
    sys.stdout.flush()
    
text = []
for k, i in enumerate(urls):
    flushPrint(k)
    content = requests.get(i)
    content.encoding = 'utf-8'
    js = content.json()
    js = js['data']['binfo']['_content']
    soup = BeautifulSoup(js, 'html.parser') 
    text.append(soup.text)

423

In [12]:
len(text)

424

In [13]:
df['content'] = text

In [14]:
df.head()

,personnel_name,pkid,proposal_number,publish_time,reason,rownum,type,year,content
0,邹正,18b1b347f9e34badb8934c2acec80e9e,0001,2019-01-12 16:04:48,关于完善城市环卫公厕指引系统的建议,1,城乡建设,2019,调研情况： 2015 年 4 月 1 日，习近平总书记首次提出要坚持不懈地推进“厕所革...
1,省政协学习委员会,da43aae2378244faa961dd1224d1343e,0002,2019-01-12 16:04:48,关于加强老小区光纤化改造的建议,2,城乡建设,2019,调研情况：近期，省政协学习委员会组织部分委员对我省信息通信业发展情况进行考察调研，总的感到，...
2,许文前,c0a1626a1bb744ebb0852cf25b21fb0a,0004,2019-01-12 15:42:19,加强科技创新，推动制造业转型升级,3,工业商贸,2019,调研情况： 早在2012年，美国国会的一份报告就声称，华为和中兴通讯可能涉嫌从事威胁美国...
3,段绪强,ce60d71296764cfe997d62bb2c0990af,0005,2019-01-12 16:21:46,关于落实金融政策、促进民营企业高质量发展的建议,4,财税金融,2019,调研情况：2018年，国家支持民营企业融资所出台的政策众多、且力度空前。这在一定程度上提振了...
4,侯建军,8b5fb5a7d86547899835a12af398ffc7,0006,2019-01-12 15:42:19,关于主基地航空公司协同东部机场集团发展的建议,5,工业商贸,2019,调研情况：2018年初，在呈报的题为《关于大力发展江苏民航补齐综合交通运输体系短板的几点建议...


In [15]:
#df.to_csv('../data/jszx2019.csv', index = False)